In [1]:
import os
import shutil
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from PIL import Image


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip "/content/drive/MyDrive/dataset_road.zip" -d "/content/dataset2"

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset2/train/Speed Limit 100/001036_jpg.rf.ca42fc70291b1202ec2a8b2c4ee883ad.jpg  
  inflating: /content/dataset2/train/Speed Limit 100/00007_00000_00022_png.rf.54a547dd927fd8796a86c1565e926031.xml  
  inflating: /content/dataset2/train/Speed Limit 100/001206_jpg.rf.8a5c5cff39957ab4aa0328ac121a8053.xml  
  inflating: /content/dataset2/train/Speed Limit 100/00007_00043_00012_png.rf.4b31a6c2f1c9573d9eaacbaf7c812053.xml  
  inflating: /content/dataset2/train/Speed Limit 100/001062_jpg.rf.577a1ed9865c7431d04f1174e8297bba.xml  
  inflating: /content/dataset2/train/Speed Limit 100/00007_00041_00004_png.rf.43ce27b2a1a3f2a063603fe6c8d80444.jpg  
  inflating: /content/dataset2/train/Speed Limit 100/road765_png.rf.e353454481811bcd2d8f0b7adfcfe35c.jpg  
  inflating: /content/dataset2/train/Speed Limit 100/00007_00041_00017_png.rf.3dd427c6546586db8c9bb3f01ead6f92.xml  
  inflating: /content/dataset2/train/Speed Limit 100/Fis

In [4]:
dataset_root = '/content/dataset2'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [5]:
class RoadSignDataset(Dataset):
    def __init__(self, root_dir, split="train", transform=None):
        self.root_dir = os.path.join(root_dir, split)
        self.transform = transform

        # Get the list of class folders (labels)
        self.class_names = sorted(os.listdir(self.root_dir))

        # Initialize lists to store image paths and corresponding labels
        self.image_paths = []
        self.labels = []

        # Iterate over class folders
        for label_idx, class_name in enumerate(self.class_names):
            class_dir = os.path.join(self.root_dir, class_name)

            # Get list of image files in the class folder
            image_files = [f for f in os.listdir(class_dir) if f.endswith('.jpg')]

            # Append image paths and corresponding labels
            self.image_paths.extend([os.path.join(class_dir, img) for img in image_files])
            self.labels.extend([label_idx] * len(image_files))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
# Define dataset and data loaders for train and validation sets
train_dataset = RoadSignDataset(root_dir=dataset_root, split='train', transform=transform)
test_dataset = RoadSignDataset(root_dir=dataset_root, split='test', transform=transform)
val_dataset = RoadSignDataset(root_dir=dataset_root, split='valid', transform=transform)

In [7]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [19]:
# Define the ResNet model
model = models.resnet18(pretrained=False)

# Define the ShuffleNetV3 model
#model = models.shufflenet_v2_x1_0(pretrained=False)

#model = models.mobilenet_v3_large(pretrained=False)


num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.class_names))

#num_ftrs = model.classifier[3].in_features
#model.classifier[3] = nn.Linear(num_ftrs, len(train_dataset.class_names))




In [20]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [21]:
# Move model to device if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [23]:
# Training and evaluation
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=50):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_accuracy = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            running_accuracy += (predicted == labels).float().mean().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = running_accuracy / len(train_loader)

        # Validation
        model.eval()
        val_running_loss = 0.0
        val_running_accuracy = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                val_loss = criterion(outputs, labels)
                val_running_loss += val_loss.item()

                _, predicted = torch.max(outputs, 1)
                val_running_accuracy += (predicted == labels).float().mean().item()

        val_loss = val_running_loss / len(val_loader)
        val_accuracy = val_running_accuracy / len(val_loader)

        print(f'Epoch {epoch + 1}/{num_epochs}, '
              f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2%}, '
              f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}')

    print('Training completed')

# Train the model
train_model(model, criterion, optimizer, train_loader, val_loader)

# Evaluate the model on test set
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Evaluate the model
evaluate_model(model, test_loader)

Epoch 1/50, Training Loss: 0.1643, Training Accuracy: 92.11%, Validation Loss: 1.0431, Validation Accuracy: 69.97%
Epoch 2/50, Training Loss: 0.1587, Training Accuracy: 92.42%, Validation Loss: 1.1022, Validation Accuracy: 69.82%
Epoch 3/50, Training Loss: 0.1565, Training Accuracy: 92.53%, Validation Loss: 0.9692, Validation Accuracy: 72.22%
Epoch 4/50, Training Loss: 0.1644, Training Accuracy: 91.97%, Validation Loss: 1.0244, Validation Accuracy: 71.75%
Epoch 5/50, Training Loss: 0.1548, Training Accuracy: 92.17%, Validation Loss: 1.0745, Validation Accuracy: 70.54%
Epoch 6/50, Training Loss: 0.1496, Training Accuracy: 92.11%, Validation Loss: 1.0639, Validation Accuracy: 70.54%
Epoch 7/50, Training Loss: 0.1594, Training Accuracy: 92.10%, Validation Loss: 1.0539, Validation Accuracy: 72.22%
Epoch 8/50, Training Loss: 0.1444, Training Accuracy: 92.54%, Validation Loss: 1.0678, Validation Accuracy: 72.10%
Epoch 9/50, Training Loss: 0.1542, Training Accuracy: 91.78%, Validation Loss: 1